In [ ]:
import os

import skimage
import skimage.io as sio
import matplotlib.pyplot as plt

import numpy as np

my_cmap = plt.get_cmap("magma")

In [ ]:
# Figure 0, autotomy examples and implied usage rates

folders = ["../assets/u1_m0_"]#, "../assets/u1_m1_", "../assets/u1_m2_", "../assets/u1_m3_"]

# solves with autotomy
solves = [["s600","s800"], \
          ["s100", "s600", "s700", "s800","s29","s31","s37","s41"],\
          ["s37","s41"],\
          ["s700","s31","s39"]]
bw = 16
for jj, folder in enumerate(folders):
    list_dir = os.listdir(folder)
    img = None
    for filepath in list_dir:
        print(filepath)
        if filepath.endswith("png"):
            temp_img = sio.imread(os.path.join(folder, filepath))[100:600, 300:-300]
            temp_img[:bw,:,:] = .0
            temp_img[:,:bw,:] = 0
            temp_img[-bw:,:,:] = 0
            temp_img[:,-bw:,:] = 0
            
            solved = False
            for kk in range(len(solves[jj])):
                if solves[jj][kk] in filepath:
                    solved = True
            
            if solved:
                print(f"solve indicated in {filepath}")
                # teal border for solve
                temp_img[:bw,:,1:3] = 255
                temp_img[:,:bw,1:3] = 255
                temp_img[-bw:,:,1:3] = 255
                temp_img[:,-bw:,1:3] = 255
            else:
                # red border for no solve
                temp_img[:bw,:,0] = 255
                temp_img[:,:bw,0] = 255
                temp_img[-bw:,:,0] = 255
                temp_img[:,-bw:,0] = 255

            if img is None:
                img = temp_img
            else:
                img = np.append(img, temp_img, axis=0)
                
    
        
        if img is not None:
            folder_name = os.path.split(folder)[-1]
            #sio.imsave(os.path.join("../assets", f"bar_{folder_name}.jpg"), img)
            
        

In [ ]:
# Figure 1 forward/reverse

fig, ax = plt.subplots(1,2, figsize=(10,5))
forward = sio.imread("../assets/forward_square_s2.png")[100:600,300:-300,:]
#forward_frame_agent0_epd0_step24.png")[100:600,300:-300,:]
reverse = sio.imread("../assets/reverse_square_s2.png")[100:600,300:-300,:]
#reverse_frame_agent0_epd0_step1033.png")[:500,300:-300,:]
print(reverse.shape)

ax[0].imshow(forward)
ax[0].set_title("Forward Reward", fontsize=22)
ax[1].imshow(reverse)
ax[1].set_title("Reverse Reward,\n Post-Autotomy", fontsize=22)
ax[1].arrow(300,50,-125,0, color="green", width=20, head_width = 40)

ax[0].arrow(300,50,125,0, color="green", width=20, head_width = 40)

for xx in range(2):
        ax[xx].set_xticklabels("")
        ax[xx].set_yticklabels("")

plt.savefig("../assets/forward_reverse.png")
plt.show()


# draw_neural_net from https://gist.github.com/riveSunder/05c7728c052f2ba28b494ad352f7e939
# adapted from craffel at https://gist.github.com/craffel/2d727968c3aaebd10359

def draw_neural_net(ax, left, right, bottom, top, layers):
    """
 
    Draw a neural network cartoon using matplotilb.
    
    :usage:
        >>> fig = plt.figure(figsize=(12, 12))
        >>> [np.random.randn(12,16),np.random.randn(16,4)]
    
    :parameters:
        - ax : matplotlib.axes.AxesSubplot
            The axes on which to plot the cartoon (get e.g. by plt.gca())
        - left : float
            The center of the leftmost node(s) will be placed here
        - right : float
            The center of the rightmost node(s) will be placed here
        - bottom : float
            The center of the bottommost node(s) will be placed here
        - top : float
            The center of the topmost node(s) will be placed here[np.random.randn(12,16),np.random.randn(16,4)]
        - layers : list of weight arrays
    """
    ax.axis('off')
    n_layers = len(layers)
    layer_sizes = [layer.shape[0] for layer in layers]
    layer_sizes.append(layers[-1].shape[1])
    
    v_spacing = (top - bottom)/float(max(layer_sizes))
    h_spacing = (right - left)/float(len(layer_sizes) - 1)
    # Nodes
    for n, layer_size in enumerate(layer_sizes):
        layer_top = v_spacing*(layer_size - 1)/2. + (top + bottom)/2.
        for m in range(layer_size):
            circle = plt.Circle((n*h_spacing + left, layer_top - m*v_spacing), v_spacing/4.,
                                color=np.ones(3)*0.9, ec='k', zorder=40)
            ax.add_artist(circle)
    my_cmap = plt.get_cmap("magma")
    # Edges
    for n, (layer_size_a, layer_size_b) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
        layer_top_a = v_spacing*(layer_size_a - 1)/2. + (top + bottom)/2.
        layer_top_b = v_spacing*(layer_size_b - 1)/2. + (top + bottom)/2.
        for m in range(layer_size_a):
            for o in range(layer_size_b):
                weight_value = layers[n][m,o]
                if weight_value:
                    wv_sign = np.sign(weight_value)
                    my_color = my_cmap(64) if weight_value < 0.0 else my_cmap(192) #'r' if weight_value < 0.0 else 'b'
                    my_color = 'k' if weight_value == 1.0 else my_color
                    line = plt.Line2D([n*h_spacing + left, (n + 1)*h_spacing + left],
                                      [layer_top_a - m*v_spacing, layer_top_b - o*v_spacing], \
                                     lw = 3*np.abs(weight_value), color=my_color, alpha = 0.75)
                    ax.add_artist(line)
                    
fig = plt.figure(figsize=(5,5))
draw_neural_net(fig.gca(), .1, .9, .1, .9, [np.random.randn(4,6),np.random.randn(6,3)])
#plt.savefig("../assets/neural_net.png")
plt.show()



In [ ]:
# Figure 2, body plan modes

# mode 0, joint evo
joint = sio.imread("../assets/joint_frame_agent0_epd0_step0001.png")[:,300:-300,:]
# mode 1, square
square = sio.imread("../assets/square_frame_agent0_epd0_step0001.png")[:,300:-300,:]
# mode 2, table
table = sio.imread("../assets/table_frame_agent0_epd0_step0001.png")[:,300:-300,:]
# mode 3, comb
comb = sio.imread("../assets/comb_frame_agent0_epd0_step0001.png")[:,300:-300,:]

fig, ax = plt.subplots(2,2, figsize=(10,10))

ax[0,0].imshow(joint)
ax[0,0].set_title("Mode 0: Evolved Body Plan", fontsize=20)
ax[0,1].imshow(square)
ax[0,1].set_title("Mode 1: Square Body Plan", fontsize=20)
ax[1,0].imshow(table)
ax[1,0].set_title("Mode 2: Table Body Plan", fontsize=20)
ax[1,1].imshow(comb)
ax[1,1].set_title("Mode 3: Comb Body Plan", fontsize=20)

for xx in range(2):
    for yy in range(2):
        ax[xx,yy].set_xticklabels("")
        ax[xx,yy].set_yticklabels("")

#plt.savefig("../assets/body_modes.png")
plt.show()



In [ ]:
# Figure 4: plots of evo curves for modes 0 through 3

import os
import sys
import argparse

import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["font.size"] = 18

folder_list = ["../results/u1_m0_","../results/u1_m1_","../results/u1_m2_","../results/u1_m3_",]
auto_used = [["s600", "s800", "s41"],\
             ["s100", "s600", "s700", "s800", "s29", "s31", "s37", "s41","s43"],\
             ["s100", "s200", "s300", "s17", "s37", "s41"],\
             ["s700", "s800", "s31", "s39","s41"]]

independent_variable = "generation"
threshold = 32
save_fig = True
plot_all = True
y_limits = [0, 49, -5, 89]
line_width = 4.5

below_solve = my_cmap(16)
above_solve = my_cmap(192)

if plot_all:
    my_fig, my_ax = plt.subplots(4,1,figsize=(7,18))
    
    run = 0
    my_cmap = plt.get_cmap("viridis")
    color_index = 0

for hh, folder in enumerate(folder_list):
    
    my_ax2 = my_ax[hh].twinx()# // 2, hh % 2]
    
    my_dir = os.listdir(folder)

    if "m0" in folder:
        my_mode = 0
    elif "m1" in folder:
        my_mode = 1
    elif "m2" in folder:
        my_mode = 2
    elif "m3" in folder:
        my_mode = 3
        
    title = f"Mode {my_mode} Evolution Curves"
    print(folder)
    auto_line = None
    no_auto_line = None
    
    for filename in my_dir:
        if "progress" in filename and ".npy" in filename:
            filepath = os.path.join(folder, filename)
            my_data = np.load(filepath, allow_pickle=True)

            my_data = my_data[np.newaxis][0]

            x = my_data[independent_variable]
            y = np.array(my_data["mean_fitness"])

            max_y = np.array(my_data["max_fitness"])
            min_y = np.array(my_data["min_fitness"])
            std_dev_y = np.array(my_data["std_dev_fitness"])

            if (1):
                fig = plt.figure(figsize=(10,8))

                plt.plot(x, y, 'k', label="Mean fitness", lw=4, alpha=0.5)
                plt.plot(x, max_y, '--r', label="Max fitness", lw=4, alpha=0.5)
                
                plt.fill_between(x, y-std_dev_y, y+std_dev_y, alpha=0.25, label="+/- standard deviation")
                
                plt.xlabel(independent_variable, fontsize=22)
                
                plt.ylabel("fitness", fontsize=22)
                
                plt.axis([y_limits[2], y_limits[3], y_limits[0]-15, y_limits[1]])
                
                plt.title(f"mode{my_mode}_{filename[-14:-4]}",\
                        fontsize=20)
                plt.legend(fontsize=22)

                 

                plt.fill_between([np.min(x)-10, np.max(x)+10], \
                        [threshold, threshold],\
                        [y_limits[1], y_limits[1]], color=above_solve,\
                        alpha=0.125)
                plt.fill_between([np.min(x)-10, np.max(x)+10], \
                        [y_limits[0]-20, y_limits[0]-20], \
                        [threshold, threshold],\
                        color=below_solve,\
                        alpha=0.125)
        
                #plt.tight_layout()

                if save_fig:
                    folder_name = os.path.split(folder)[-1]
                    fig.savefig(os.path.join("../assets","all_curves",folder_name + filename[:-4] + ".png"))

            if plot_all:
                
                #use_color = my_cmap(32)
                color_index = 32
                
                for jj in range(len(auto_used[hh])):
                    if auto_used[hh][jj] in filename:
                        color_index = 192
                        
                use_color = my_cmap(color_index) 
                
                if color_index == 32:    
                    if no_auto_line is None:
                        no_auto_line = my_ax[hh].plot(x, max_y, color=use_color, \
                                lw=line_width, alpha=0.35, label="autotomy not used")
                    else:
                        my_ax[hh].plot(x, max_y, color=use_color, lw=line_width, alpha=0.35)
                        
                if color_index == 192:
                    if auto_line is  None:
                        auto_line = my_ax[hh].plot(x, max_y, color=use_color, \
                                lw=line_width, alpha=0.35, label="autotomy used")
                    else:
                        my_ax[hh].plot(x, max_y, color=use_color, lw=line_width, alpha=0.35)

                
                my_ax2.axis([y_limits[2], y_limits[3], 0, 0.49])
                my_ax[hh].plot(x[-1], max_y[-1], "o",color=use_color, ms=6, alpha=0.35)
                my_ax[hh].axis([y_limits[2], y_limits[3], y_limits[0], y_limits[1]])               


                run += 1
                my_ax[hh].legend() 

    if plot_all:
        #my_ax.plot([np.min(x)-10, np.max(x)+10], [threshold for e in range(2)], "k", ".-", lw=6, alpha=0.25, label="Solution Threshold")

        my_ax[hh].fill_between([np.min(x)-10, np.max(x)+10], \
                [y_limits[0], y_limits[0]], \
                [threshold, threshold],\
                color=below_solve,\
                alpha=0.125)

        my_ax[hh].fill_between([np.min(x)-10, np.max(x)+10], \
                [threshold, threshold],\
                [y_limits[1], y_limits[1]], color=above_solve,\
                alpha=0.125, label="Solution Threshold")

        my_ax[hh].set_ylim([y_limits[0], y_limits[1]])
        my_ax[hh].set_xlim([np.min(x)-5, np.max(x)+5])
        my_ax[hh].set_title(title.replace(",", "\n"), fontsize=32)

        my_ax[hh].set_xlabel(independent_variable, fontsize=22)
        my_ax[hh].set_ylabel("best agent fitness", fontsize=22)
        
        my_fig.tight_layout()

if save_fig:

    folder_name = os.path.split(folder)[-1]
    my_fig.savefig(os.path.join("../assets", \
            folder_name + filename[:-4] + "all.png"))

plt.show()
        


In [ ]:
fn_auto_forward = "../supplemental_information/assets/m1_s600_forward.png"
fn_auto_reverse = "../supplemental_information/assets/m1_s600_reverse.png"

fn_noto_forward = "../supplemental_information/assets/m0_s700_forward.png"
fn_noto_reverse = "../supplemental_information/assets/m0_s700_reverse.png"

fn_log_auto = "../supplemental_information/results/u1_m1_/progress_BackAndForthEnv-v0_Geneti1665112701_s600.npy"
fn_log_noto = "../supplemental_information/results/u1_m0_/progress_BackAndForthEnv-v0_Geneti1665144148_s700.npy"

auto_forward = sio.imread(fn_auto_forward)[:,300:-300,:]
auto_reverse = sio.imread(fn_auto_reverse)[:,300:-300,:]

noto_forward = sio.imread(fn_noto_forward)[:,300:-300,:]
noto_reverse = sio.imread(fn_noto_reverse)[:,300:-300,:]

fig, ax = plt.subplots(3,2, figsize=(12,18))

ax[0,0].set_title("Solve with autotomy,\n mode 1", fontsize=32)
ax[0,0].set_ylabel("Forward", fontsize=32)
ax[1,0].set_ylabel("Reverse", fontsize=32)
ax[0,0].imshow(auto_forward)
ax[1,0].imshow(auto_reverse)

ax[0,1].set_title("Solve, no autotomy,\n mode 0", fontsize=32)
ax[0,1].imshow(noto_forward)
ax[1,1].imshow(noto_reverse)
for xx in range(2):
    for yy in range(2):
        ax[xx,yy].set_xticklabels("")
        ax[xx,yy].set_yticklabels("")

y_limits = [0,64, 0, 85]

for ll, filepath in enumerate([fn_log_auto, fn_log_noto]):
    
    my_data = np.load(filepath, allow_pickle=True)

    my_data = my_data[np.newaxis][0]

    x = my_data[independent_variable]
    y = np.array(my_data["mean_fitness"])

    max_y = np.array(my_data["max_fitness"])
    min_y = np.array(my_data["min_fitness"])
    std_dev_y = np.array(my_data["std_dev_fitness"])
    ax[2,ll].plot(x, y, 'k', label="Mean fitness", lw=4, alpha=0.5)
    ax[2,ll].plot(x, max_y, '--r', label="Max fitness", lw=4, alpha=0.5)

    ax[2,ll].fill_between(x, y-std_dev_y, y+std_dev_y, alpha=0.25, label="+/- std. dev.")

    ax[2,ll].set_xlabel(independent_variable, fontsize=28)


    ax[2,ll].axis([y_limits[2], y_limits[3], y_limits[0]-15, y_limits[1]])

    
    if ll == 0:
        ax[2,ll].legend(fontsize=18, loc='upper left')
        
        ax[2,ll].set_ylabel("fitness", fontsize=28)
        ax[2,ll].set_title(f"mode 1 seed 600",\
            fontsize=32)
    else:
        ax[2,ll].set_title(f"mode 0 seed 700",\
            fontsize=32)


    ax[2,ll].fill_between([np.min(x)-10, np.max(x)+10], \
            [threshold, threshold],\
            [y_limits[1], y_limits[1]], color=above_solve,\
            alpha=0.125)
    ax[2,ll].fill_between([np.min(x)-10, np.max(x)+10], \
            [y_limits[0]-20, y_limits[0]-20], \
            [threshold, threshold],\
            color=below_solve,\
            alpha=0.125)
    
#plt.tight_layout()
plt.savefig("../assets/solve_examples.png")
plt.show()

In [ ]:
# Figure 6 joint evo body plans collage

auto_folder = "../assets/u1_m0_/f_r_autotomy"

noto_folder = "../assets/u1_m0_/no_autotomy"

noto_files = os.listdir(noto_folder)

collage = np.ones((2400,2400,4))
collage[1800:2400,1200:2400,3] = 0

bw = 16

for cc, filename in enumerate(noto_files):
    
    img = sio.imread(os.path.join(noto_folder, filename))[:,300:-300,:] / 255.
    
    print(cc, filename)
    
    x_coord = (cc // 4) * 600
    y_coord = (cc % 4) * 600
    img[:bw,:,:] = 0.05
    img[:,:bw,:] = 0.05
    img[-bw:,:,:] = .05
    img[:,-bw:,:] = 0.05
    
    collage[x_coord:x_coord+600, y_coord:y_coord+600,:3] = img
    
bar = np.ones((1800,600,4))

auto_forward = ["m0_s600_forward.png", "m0_s800_forward.png", "m0_s41_forward.png"]

for dd, auto in enumerate(auto_forward):
    
    img = sio.imread(os.path.join(auto_folder, auto))[:,300:-300,:] / 255.
    
    img[:bw,:,:] = 0.05
    img[:,:bw,:] = 0.05
    img[-bw:,:,:] = .05
    img[:,-bw:,:] = 0.05
    
    bar[600*dd:600*(dd+1),:,:3] = img

    
plt.figure()
plt.imshow(collage )
plt.figure()
plt.imshow(bar)


bar = np.array(bar * 255, dtype=np.uint8)

collage = np.array(collage * 255, dtype=np.uint8)
sio.imsave("../assets/m0_bar.png", bar)
sio.imsave("../assets/m0_collage.png", collage)

In [ ]:
m0a_sizes = [4,6,8,6,2,2,8,4,3,14,3,3,2,2,2,4,2]
m0b_sizes = [4,6,8,6,2,2,8,4,3,14,3,3,2,2,1,1,1]
m1_sizes = [3,4,5,4,9,8,8,5,10] 
m2_sizes = [4,3,4,6,5] 
m3_sizes = [3,4,3,7,4]

m1a_sizes = [3,4,5,4,9,8,8,5,10] + 8 * [25]
m2a_sizes = [4,3,4,6,5] + 12*[19]
m3a_sizes = [3,4,3,7,4] + 12*[13]

m0a_mean = np.mean(m0a_sizes)
m0b_mean = np.mean(m0b_sizes)

m1_mean = np.mean(m1_sizes)
m2_mean = np.mean(m2_sizes)
m3_mean = np.mean(m3_sizes)

m0a_std = np.std(m0a_sizes)
m0b_std = np.std(m0b_sizes)

m1_std = np.std(m1_sizes)
m2_std = np.std(m2_sizes)
m3_std = np.std(m3_sizes)

m1a_mean = np.mean(m1a_sizes)
m2a_mean = np.mean(m2a_sizes)
m3a_mean = np.mean(m3a_sizes)
m1a_std = np.std(m1a_sizes)
m2a_std = np.std(m2a_sizes)
m3a_std = np.std(m3a_sizes)

all_auto = m1_sizes + m2_sizes + m3_sizes + [1] * 3

all_auto_mean = np.mean(all_auto)
all_auto_std = np.std(all_auto)
m1_3_auto_mean = np.mean(m1_sizes + m2_sizes + m3_sizes)
m1_3_auto_std = np.std(m1_sizes + m2_sizes + m3_sizes)

print(f"mode 0a average body size = {m0a_mean:.4f} +/- {m0a_std:.4f}")
print(f"mode 0b average body size = {m0b_mean:.4f} +/- {m0b_std:.4f}")
print(f"mode 1 auto only average body size = {m1_mean:.4f} +/- {m1_std:.4f}")
print(f"mode 2 auto only average body size = {m2_mean:.4f} +/- {m2_std:.4f}")
print(f"mode 3 auto only average body size = {m3_mean:.4f} +/- {m3_std:.4f}")


print(f"mode 1 including nototomy average body size = {m1a_mean:.4f} +/- {m1a_std:.4f}")
print(f"mode 2 including nototomy average body size = {m2a_mean:.4f} +/- {m2a_std:.4f}")
print(f"mode 3 including nototomy average body size = {m3a_mean:.4f} +/- {m3a_std:.4f}")


print(f"all modes after autotomy average body size = {all_auto_mean:.4f} +/- {all_auto_std:.4f}")
print(f"all modes less mode 0 after autotomy average body size = {m1_3_auto_mean:.4f} +/- {m1_3_auto_std:.4f}")

In [ ]:
mode0 = np.array([\
        41.02,\
        27.34,\
        41.70,\
        41.11,\
        40.47,\
        41.77,\
        41.13,\
        41.88,\
        31.12,\
        31.00,\
        41.34,\
        17.91,\
        24.56,\
        39.26,\
        43.24,\
         21.92,\
         24.21,\
        ])

mode0_auto = np.array([\
             0,\
             0,\
             0,\
             0,\
             0,\
             1,\
             0,\
             1,\
             0,\
             0,\
             0,\
             0,\
             0,\
             0,\
             0,\
             1,\
             0,\
             ])

mode1_auto = np.array([\
                      1,\
                      0,\
                      0,\
                      0,\
                      0,\
                      1,\
                      1,\
                      1,\
                      0,\
                      0,\
                      0,\
                      1,\
                      1,\
                      1,\
                      0,\
                      1,\
                      1\
                      ])
mode2_auto = np.array([\
                       1,\
                       1,\
                       0,\
                       0,\
                       0,\
                       0,\
                       0,\
                       0,\
                       1,\
                       0,\
                       0,\
                       0,\
                       0,\
                       1,\
                       0,\
                       1,\
                       0,\
                      ])
mode3_auto = np.array([\
                      0,\
                      0,\
                      0,\
                      0,\
                      0,\
                      0,\
                      1,\
                      1,\
                      0,\
                      0,\
                      0,\
                      0,\
                      1,\
                      0,\
                      1,\
                      1,\
                      0])
mode1 = np.array([\
                  41.68,\
                  33.87,\
                  43.09,\
                  28.74,\
                  40.14,\
                  40.87,\
                  32.53,\
                  41.59,\
                  41.86,\
                  42.86,\
                  33.24,\
                  39.42,\
                  40.73,\
                  42.92,\
                  40.60,\
                  39.08,\
                  11.52\
                 ])
                 
mode2 = np.array([\
                 26.0,\
                  14.69,\
                  31.78,\
                  29.50,\
                  41.48,\
                  41.13,\
                  41.94,\
                  33.93,\
                  19.91,\
                  31.60,\
                  42.48,\
                  27.14,\
                  28.56,\
                  32.48,\
                  41.05,\
                  41.55,\
                  42.60])
                 
mode3 = np.array([\
                  25.13,\
                  41.35,\
                  42.51,\
                  32.49,\
                  40.43,\
                  23.58,\
                  38.38,\
                  21.30,\
                  25.57,\
                  33.35,\
                  40.48,\
                  26.10,\
                  40.96,\
                  42.30,\
                  41.85,\
                  26.92,\
                  38.70])

m0_m = np.mean(mode0)
m0_sd = np.std(mode0)

m0_m_a = np.mean(mode0[mode0_auto == 1])
m0_sd_a = np.std(mode0[mode0_auto == 1])

m0_m_n = np.mean(mode0[mode0_auto == 0])
m0_sd_n = np.std(mode0[mode0_auto == 0])

m1_m = np.mean(mode1)
m1_sd = np.std(mode1)

m1_m_a = np.mean(mode1[mode1_auto == 1])
m1_sd_a = np.std(mode1[mode1_auto == 1])

m1_m_n = np.mean(mode1[mode1_auto == 0])
m1_sd_n = np.std(mode1[mode1_auto == 0])


m2_m = np.mean(mode0)
m2_sd = np.std(mode0)

m2_m_a = np.mean(mode2[mode2_auto == 1])
m2_sd_a = np.std(mode2[mode2_auto == 1])

m2_m_n = np.mean(mode2[mode2_auto == 0])
m2_sd_n = np.std(mode2[mode2_auto == 0])

m3_m = np.mean(mode3)
m3_sd = np.std(mode3)

m3_m_a = np.mean(mode3[mode3_auto == 1])
m3_sd_a = np.std(mode3[mode3_auto == 1])

m3_m_n = np.mean(mode3[mode3_auto == 0])
m3_sd_n = np.std(mode3[mode3_auto == 0])




fitness_table = "\\begin{table}[h] \n"
fitness_table +=    "\\center{ \n"
fitness_table +="\\begin{tabular}{l | c | c | c |}\n"
fitness_table +="Mode & Fitness $\pm$ std. dev. & with autotomy &  without autotomy \\ \n"
fitness_table +="\\hline \n"
fitness_table += f"Mode 0 & {m0_m:.3} $\\pm$ {m0_sd:.3} & {m0_m_a:.3} $\\pm$ {m0_sd_a:.3} & {m0_m_n:.3} $\\pm$ {m0_sd_n:.3} \\\\\n"
fitness_table += f"Mode 1 & {m1_m:.3} $\\pm$ {m1_sd:.3} & {m1_m_a:.3} $\\pm$ {m1_sd_a:.3} & {m1_m_n:.3} $\\pm$ {m1_sd_n:.3} \\\\\n"
fitness_table += f"Mode 2 & {m2_m:.3} $\\pm$ {m2_sd:.3} & {m2_m_a:.3} $\\pm$ {m2_sd_a:.3} & {m2_m_n:.3} $\\pm$ {m2_sd_n:.3} \\\\\n"
fitness_table += f"Mode 3 & {m3_m:.3} $\\pm$ {m3_sd:.3} & {m3_m_a:.3} $\\pm$ {m3_sd_a:.3} & {m3_m_n:.3} $\\pm$ {m3_sd_n:.3} \\\\\n"
fitness_table +="    \\end{tabular}\n"
fitness_table +="    }\n"
fitness_table +="    \\caption{\n"
fitness_table +="    Mean fitness $\\pm$ standard deviation across evolution runs for each mode, with and without autotomy. \n"
fitness_table +="    }\n"
fitness_table +="    \\label{table:with_without}\n"
fitness_table +="\\end{table}\n"

print(fitness_table)

In [ ]:

rates_table = """
\\begin{table}[h]
\\center{ 
\\begin{tabular}{l | c | c | c | c }
Mode & Solve rate &  Autotomy usage rate & P(a|s) &  P(s|a) \\\\
\\hline
"""

rates_table += f"Mode 0 & {10./17:.3} & {3./17:.3}  & {2./10:.3} & {2./3:.3}  \\\\ \n"
rates_table += f"Mode 1 & {15./17:.3} & {9./17:.3}  & {8./15:.3} & {8./9:.3}  \\\\ \n"
rates_table += f"Mode 2 & {9./17:.3}  & {5./17:.3}  & {2./9:.3}  & {2./5:.3}  \\\\ \n"
rates_table += f"Mode 3 & {11./17:.3} & {5./17:.3}  & {3./11:.3} & {2./5:.3}  \\\\ \n"

rates_table += """
    \\end{tabular}
    }
    \\caption{
    Rates of task solution and autotomy usage across evolution runs with 17 random seeds.
    }
    \\label{table:solve_rates}
\\end{table}
"""

print(rates_table)